### Importing libraries, setting Logging level and Constant values

In [173]:
import io
import json
import logging

import boto3
import pandas as pd
import requests
import zipfile

from botocore.exceptions import ClientError

# Setting LOG level and format, for mor info visit https://www.logicmonitor.com/blog/python-logging-levels-explained
logging.basicConfig(format='%(levelname)s : %(message)s',
                    level=logging.INFO)

# Values to change according your own configs
PROFILE_NAME =  "dsa-gepp"
S3_BUCKET = 'my-s3-bucket-john'
FILE_NAME = "denue_inegi_09_.csv"
URL = 'https://www.inegi.org.mx/contenidos/masiva/denue/denue_09_csv.zip'

# Constant values
SECRET_NAME = "airflow/connections/dev/rds-mysql"
REGION_NAME = "us-east-1"
S3_KEY_ORIGINAL = f"session-14-airflow/denue/original/{FILE_NAME}"
S3_KEY_FINAL = f"session-14-airflow/denue/final/{FILE_NAME}"

### Getting the AWS RDS MySQL credentials

In [184]:
def get_secret(secret, region, profile="default"):
    """
    Get the secret from AWS Secrets Manager
    
    :param secret (str): Name of the Secret
    :param region (str): Region in which the Secret is stored
    :param profile (str): Profile name to be executed, left in blank to take default (optional)
    
    :return: String with the Secret Value
    """    
    
    secret_name = secret
    region_name = region

    # Create a Secrets Manager client
    session = boto3.session.Session(profile_name=profile)
    client = session.client(
        service_name='secretsmanager',
        region_name=region
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])

    return secret

In [180]:
# Testing
print(get_secret(SECRET_NAME, REGION_NAME, PROFILE_NAME))

INFO : Found credentials in shared credentials file: ~/.aws/credentials


{'username': 'admin', 'password': 'dataGepp5', 'engine': 'mysql', 'host': 'data-course-wize.c2jptj5kvukd.us-east-1.rds.amazonaws.com', 'port': 3306, 'dbname': 'datawzcourse', 'dbInstanceIdentifier': 'data-course-wize'}


### Method to upload a file to an S3 Bucket

In [155]:
def upload_file_to_s3(bucket_name, s3_key, file_object, region, profile="default"):
    """
    Upload a file to an S3 bucket
    
    :param file_path (str): Local file path of the file to upload
    :param bucket_name (str): Name of the S3 bucket to upload the file to
    :param s3_key (str): S3 key (i.e. object name) to assign to the file in the bucket
    :param region (str): Region in which the Secret is stored
    :param profile (str): Profile name to be executed, left in blank to take default (optional)
    
    :return: botocore.response object. If error, returns None.
    """
    
    # Create S3 client
    session = boto3.session.Session(profile_name=profile)
    s3_client = session.client(
        service_name='s3',
        region_name=region
    )
    
    try:
        # Upload file to S3 bucket from local
        # s3_client.upload_file(file_path, bucket_name, s3_key)
        
        # Upload file to S3 bucket from memory (Body)
        response = s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=file_object)
        
        logging.info(f"Successfully uploaded {s3_key} to S3 bucket {bucket_name}")
    
    except Exception as e:
        logging.info(f"Error uploading file to S3: {e}")
        raise e
    
    return response


### Method to get a file to an S3 Bucket

In [156]:
def get_s3_object(bucket_name, object_name, region, profile="default"):
    """
    Retrieve an object from an S3 bucket
    
    :param bucket_name: string
    :param object_name: string
    
    :return: botocore.response object. If error, returns None.
    """
    # Create S3 client
    session = boto3.session.Session(profile_name=profile)
    s3_client = session.client(
        service_name='s3',
        region_name=region
    )
    
    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=object_name)
        logging.info(f"Successfully get {object_name} from S3 bucket {bucket_name}")
        
    except ClientError as e:
        logging.info(f"Error getting object {object_name} from bucket {bucket_name}: {e}")
        raise e
    
    return response


### Extracting/Downloading the Data (Denue)

In [157]:
import requests
import zipfile
import io

def download_and_upload_to_s3(bucket_name, s3_key, url):
    """
    Download the data from the external URL and upload it 
    decompressed to s3
    
    :param bucket_name (str): Bucket name
    :param s3_key (str): S3 key that includes the whole path with the file name
    :param url (str): URL of the external data
    
    :return: botocore.response object. If error, returns None.
    """  
    
    try:
        # Download file from URL
        response = requests.get(url)

        # Extract CSV file from ZIP archive
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            csv_filename = [f for f in z.namelist() if f.endswith(s3_key.split('/')[-1])][0]
            csv_file = z.read(csv_filename)

        # Uploading the data to s3
        response = upload_file_to_s3(bucket_name, s3_key, csv_file, REGION_NAME, PROFILE_NAME)

        return response
        
    except requests.exceptions.RequestException as e:
        logging.info(f"Failed to download data from {url}: {e}")
        raise e
        
    except zipfile.BadZipFile as e:
        logging.info(f"Failed to extract CSV file from ZIP archive: {e}")
        raise e
        
    except boto3.exceptions.Boto3Error as e:
        logging.info(f"Failed to upload {key_name} to S3 bucket {bucket_name}: {e}")
        raise e

    

In [81]:
response = requests.get(URL)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    csv_filename = [f for f in z.namelist() if f.endswith(S3_KEY.split('/')[-1])][0]
    print(csv_filename)
    #csv_file = z.read(csv_filename)

conjunto_de_datos/denue_inegi_09_.csv


In [82]:
# Testing
download_and_upload_to_s3(S3_BUCKET, S3_KEY_ORIGINAL, URL)
#print(URL)
#https://www.inegi.org.mx/contenidos/masiva/denue/denue_09_csv.zip

INFO : Found credentials in shared credentials file: ~/.aws/credentials
INFO : Successfully uploaded session-14-airflow/denue/original/denue_inegi_09_.csv to S3 bucket my-s3-bucket-john


True

In [ ]:
import pandas as pd

session = boto3.session.Session(profile_name='dsa-gepp')
s3_client = session.client(
    service_name='s3',
    region_name='us-east-1'
)

# Define the S3 bucket and file path
bucket_name = 'my-s3-bucket-john'
file_path = 'session-14-airflow/denue/original/denue_inegi_09_.csv'

# Read the CSV file from S3 using Pandas
s3_file = s3_client.get_object(Bucket=bucket_name, Key=file_path)
df_denue = pd.read_csv(s3_file['Body'], encoding='latin-1')

df_denue.head()



### Downloading Locally the data (because there is an issue with the encode when we use the s3_client)

In [158]:
def get_body_content(bucket_name, s3_key):
    """
    Gets the object.
    
    :param bucket_name (str): Bucket name
    :param s3_key (str): S3 key that includes the whole path with the file name

    :return: The object data in as a dataframe with correct column names
    """
    try:

        response = get_s3_object(bucket_name, s3_key, REGION_NAME, PROFILE_NAME)
        status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")
        logging.info(status)
        
        # Writing the file to the local file system
        with open(f"/tmp/{FILE_NAME}",'wb') as output_file:
            output_file.write(response.get("Body").read())
        
        logging.info(f'Download file /tmp/{FILE_NAME} completed')
        
        if status == 200:
            # Get columns for df
            df = pd.read_csv(f"/tmp/{FILE_NAME}", encoding='latin-1')
            return df
        else:
            raise ClientError

    except ClientError as e:
        logging.info("Couldn't get object from bucket .")
        raise e
    except Exception as e:
        logging.info("Error when creating dataframe")
        logging.info(e)
        raise e

### Cleaning, transforming and uploading the data

In [159]:
def clean_data(df_data, bucket_name, s3_key):
    # Filter by activities
    options = [461110,461111,461112,461113]
    gepp_data = df_data[df_data["codigo_act"].isin(options)]
    
    # cod_postal column should always have 5 digits
    gepp_data = gepp_data.astype({'cod_postal': 'string'})
    gepp_data["cod_postal"] = gepp_data["cod_postal"].str.replace(".0",'' , regex=False)
    gepp_data.dropna(subset=['cod_postal'], inplace=True)
    gepp_data["cod_postal"] = gepp_data["cod_postal"].apply(lambda x: x if len(x) == 5 else '0'+ x)
    
    # The entity is always Mexico City and the key is always 9
    gepp_data["is_city_valid"] = gepp_data.apply(lambda row : True if  row['cve_ent'] == 9 and row['entidad'] == "CIUDAD DE MÉXICO" else False , axis=1)    
    final_data = gepp_data[gepp_data["is_city_valid"]]
    final_data.drop(columns="is_city_valid", inplace=True)
    
    # Uploading the data to s3
    response = upload_file_to_s3(bucket_name, s3_key, final_data.to_csv(), REGION_NAME, PROFILE_NAME)
    
    if response.get("ResponseMetadata", {}).get("HTTPStatusCode") == 200:
        return True
    else:
        return False

In [160]:
# Testing
df_denue = get_body_content(S3_BUCKET, S3_KEY_ORIGINAL)
#df_denue.head()

clean_data(df_denue, S3_BUCKET, S3_KEY_FINAL)

INFO : Found credentials in shared credentials file: ~/.aws/credentials
INFO : Successfully get session-14-airflow/denue/original/denue_inegi_09_.csv from S3 bucket my-s3-bucket-john
INFO : 200
INFO : Download file /tmp/denue_inegi_09_.csv completed
/var/folders/xt/xyzrgq412p51pshkvhsj0b2m0000gp/T/ipykernel_25128/3001233026.py:24: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"/tmp/{FILE_NAME}", encoding='latin-1')
INFO : Found credentials in shared credentials file: ~/.aws/credentials
INFO : Successfully uploaded session-14-airflow/denue/final/denue_inegi_09_.csv to S3 bucket my-s3-bucket-john


True

### Python code to upload the data from s3 to Mysql(RDS)

In [181]:
import pymysql

def upload_csv_to_mysql(table_name):
    # Getting the data curated
    df = pd.read_csv(f"/tmp/{FILE_NAME}", encoding='latin-1')
    
    # Getting the MySQL credentials
    mysql_credentials = get_secret(SECRET_NAME, REGION_NAME, PROFILE_NAME)
    
    # Connect to MySQL database
    conn = pymysql.connect(
        host=mysql_credentials['host'],
        user=mysql_credentials['username'],
        password=mysql_credentials['password'],
        port=mysql_credentials['port'],
        cursorclass=pymysql.cursors.DictCursor
    )
    
    # Connect to the database
    conn.select_db(mysql_credentials['dbname'])
    
    # Create table from dataframe schema
    with conn.cursor() as cursor:
        columns = ['`{}` VARCHAR(255) NOT NULL'.format(col) for col in df.columns]
        query = 'CREATE TABLE IF NOT EXISTS `{}` ({}) ENGINE=InnoDB'.format(table_name, ','.join(columns))
        cursor.execute(query)
    
    # Load data into MySQL table
    with conn.cursor() as cursor:
        # Truncate table if exists
        cursor.execute(f"TRUNCATE TABLE `{table_name}`")
        # Create insert statement
        placeholders = ', '.join(['%s'] * len(df.columns))
        columns = ', '.join(['`{}`'.format(col) for col in df.columns])
        query = 'INSERT INTO `{}` ({}) VALUES ({})'.format(table_name, columns, placeholders)
        # Execute the insert statement for each row
        for row in df.itertuples(index=False):
            cursor.execute(query, row)
    conn.commit()
    conn.close()


In [ ]:
upload_csv_to_mysql('test_table')

## Puting all together

### Extract function

In [187]:
def extract_denue_inegi(**kwargs):
    """
    This function will extract the data from Inegi
    then will be decompressed and uploaded in csv files
    into a S3 bucket (Original)
    """

    bucket_name = kwargs['bucket_name']
    s3_key = kwargs['s3_key']
    url = kwargs['url']
    
    download_and_upload_to_s3(bucket_name, s3_key, url)

In [188]:
# Testing
op_kwargs={
        'bucket_name': S3_BUCKET, 
        's3_key': S3_KEY_ORIGINAL, 
        'url': URL                
    }

extract_denue_inegi(**op_kwargs)

INFO : Found credentials in shared credentials file: ~/.aws/credentials
INFO : Successfully uploaded session-14-airflow/denue/original/denue_inegi_09_.csv to S3 bucket my-s3-bucket-john


### Transform Function

In [ ]:
def transform_denue_inegi(**kwargs):
    """
    This function will transform the data from Inegi
    will take the data stored in S3, then download it 
    locally and perform some transformations and at the
    end the data will be stored in S3 (Final)
    """

    bucket_name = kwargs['bucket_name']
    s3_key_original = kwargs['s3_key_original']
    s3_key_final = kwargs['s3_key_final']
    
    df_denue = get_body_content(bucket_name, s3_key_original)
    clean_data(df_denue, bucket_name, s3_key_final)

In [ ]:
# Testing
op_kwargs={
        'bucket_name': S3_BUCKET, 
        's3_key_original': S3_KEY_ORIGINAL, 
        's3_key_final': S3_KEY_FINAL                
    }

transform_denue_inegi(**op_kwargs)